# A Cantera Simulation Using RMG-Py

In [ ]:
from IPython.display import display, Image

from rmgpy.chemkin import load_chemkin_file
from rmgpy.tools.canteramodel import Cantera, get_rmg_species_from_user_species
from rmgpy.species import Species

Load the species and reaction from the RMG-generated chemkin file `chem_annotated.inp` and `species_dictionary.txt` file found in your `chemkin` folder after running a job.

In [ ]:
species_list, reaction_list = load_chemkin_file('data/ethane_model/chem_annotated.inp',
                                                'data/ethane_model/species_dictionary.txt',
                                                'data/ethane_model/tran.dat')

Set a few conditions for how to react the system

In [ ]:
# Find the species: ethane
user_ethane=Species().from_smiles('CC')
species_dict = get_rmg_species_from_user_species([user_ethane], species_list)
ethane = species_dict[user_ethane]

reactor_type_list = ['IdealGasReactor']
mol_frac_list=[{ethane: 1}]
Tlist = ([1300, 1500, 2000], 'K')
Plist = ([1], 'bar')
reaction_time_list = ([0.5], 'ms')

In [ ]:
# Create cantera object, loading in the species and reactions
job = Cantera(species_list=species_list, reaction_list=reaction_list, output_directory='temp')
# The cantera file must be created from an associated chemkin file

# We can either load the Model from the initialized set of rmg species and reactions
job.load_model()

# Or load it from a chemkin file by uncommenting the following line:
#job.load_chemkin_model('data/ethane_model/chem_annotated.inp',transport_file='data/ethane_model/tran.dat')

# Generate the conditions based on the settings we declared earlier
job.generate_conditions(reactor_type_list, reaction_time_list, mol_frac_list, Tlist, Plist)

In [ ]:
# Simulate and plot
alldata = job.simulate()
job.plot(alldata)

# Show the plots in the ipython notebook
for i, condition in enumerate(job.conditions):
    print('Condition {0}'.format(i+1))
    display(Image(filename="temp/{0}_mole_fractions.png".format(i+1)))

In [ ]:
# We can get the cantera model Solution's species and reactions
ct_species = job.model.species()
ct_reactions = job.model.reactions()

# We can view a cantera species or reaction object from this
ct_ethane = ct_species[4]
ct_rxn = ct_reactions[0]

print(ct_ethane)
print(ct_rxn)

In [ ]:
# We can also do things like modifying the cantera species thermo and reaction kinetics through modifying the
# RMG objects first, then using the `modifyReactionKinetics` or `modifySpeciesThermo` functions

# Alter the RMG objects in place, lets pick ethane and the first reaction
rmg_ethane = species_dict[user_ethane]
rmg_ethane.thermo.change_base_enthalpy(2*4184)  # Change base enthalpy by 2 kcal/mol

rmg_rxn = reaction_list[0]
rmg_rxn.kinetics.change_rate(4)  # Change A factor by multiplying by a factor of 4

# Take a look at the state of the cantera model before and after
print('Cantera Model: Before')
ct_species = job.model.species()
ct_reactions = job.model.reactions()
print('Ethane Thermo = {} kcal/mol'.format(ct_species[4].thermo.h(300)/1000/4184))
print('Reaction 1 Kinetics = {}'.format(ct_reactions[0].rate))

# Now use the altered RMG objects to modify the kinetics and thermo
job.modify_reaction_kinetics(0, rmg_rxn)
job.modify_species_thermo(4, rmg_ethane, use_chemkin_identifier = True)

# If we modify thermo, the cantera model must be refreshed.  If only kinetics are modified, this does not need to be done.
job.refresh_model()

print('')
print('Cantera Model: After')
ct_species = job.model.species()
ct_reactions = job.model.reactions()
print('Ethane Thermo = {} kcal/mol'.format(ct_species[4].thermo.h(300)/1000/4184))
print('Reaction 1 Kinetics = {}'.format(ct_reactions[0].rate))

In [ ]:
# Simulate and plot
alldata = job.simulate()
job.plot(alldata)

# Show the plots in the ipython notebook
for i, condition in enumerate(job.conditions):
    print('Condition {0}'.format(i+1))
    display(Image(filename="temp/{0}_mole_fractions.png".format(i+1)))